# eLTL monitor with Apache Flink API's in Scala
## by Adnan Bouaouda Arafa, Sept. 2021, UMA Málaga.

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import $ivy.`org.apache.flink::flink-clients:1.10.3`
import $ivy.`org.apache.flink::flink-scala:1.10.3`
import $ivy.`org.apache.flink::flink-streaming-scala:1.10.3`
import $ivy.`org.scalaz::scalaz-core:7.3.5`
import $ivy.`com.lihaoyi::os-lib:0.2.7`
import $ivy.`org.apache.flink:flink-runtime-web_2.12:1.11.2`
import $ivy.`com.storm-enroute::scalameter-core:0.18`

import $ivy.$                                  

import $ivy.$                                       

import $ivy.$                                     

import $ivy.$                                               

import $ivy.$                              

import $ivy.$                          

import $ivy.$                                               

import $ivy.$                                        

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [3]:
import org.apache.flink.api.scala._
import org.apache.flink.api.scala.utils._
import org.apache.flink.util.Collector
import javax.script._
import scala.annotation.tailrec
import scalaz._
import Scalaz._
import org.scalameter._

import org.apache.flink.api.scala._

import org.apache.flink.api.scala.utils._

import org.apache.flink.util.Collector

import javax.script._

import scala.annotation.tailrec

import scalaz._

import Scalaz._

import org.scalameter._

In [4]:
import org.apache.flink.configuration.{ConfigConstants, Configuration,RestOptions}
val config = new Configuration()
val benv = ExecutionEnvironment.createLocalEnvironmentWithWebUI(config)

import org.apache.flink.configuration.{ConfigConstants, Configuration,RestOptions}

config: Configuration = {rest.port=8081}
benv: ExecutionEnvironment = org.apache.flink.api.scala.ExecutionEnvironment@5e84fcb5

In [5]:
/**
 * Implements the "eLTL" logic
 */

type T  = (Long, Int)
def memoizeFnc[K, V](f: K => V): K => V = {
val cache = collection.mutable.Map.empty[K, V]
    k =>
        cache.getOrElse(k, {
        cache update(k, f(k))
        cache(k)
        })
}

def apply[T](b: DataSet[(Long, T)], cond: T => Boolean): DataSet[(Long, Boolean)] 
        =  b.map{ e => (e._1, cond(e._2))}
val mApply = Memo.immutableHashMapMemo{apply _ tupled}

def parse[T](b: DataSet[(Long, T)], cond: T => Boolean): List[Long]
    = { lazy val lazy_b = apply(b, cond).filter{_._2}.map { pair => pair._1 }.collect.toList
        lazy_b
    }
            
def intervalPQ(lp: List[Long], lq: List[Long],  ti: Long = 0, tf: Long = Long.MaxValue): List[(Long, Long)] = {
try{    
    @tailrec def tIntervslPQ(Lp: List[Long], Lq: List[Long], acc: List[(Long, Long)] = List.empty[(Long, Long)]):List[(Long, Long)] = {
        val  Lqq = Lq.dropWhile(_ <= Lp.head)
        val  Lpp = Lp.dropWhile(_ <= Lqq.head)
        (Lpp, Lqq) match {
            case (List(), _) => acc ++ List((Lp.head, Lqq.head))
            case (_, List()) => acc ++ List((Lp.head, Lqq.head))
            case (_, _)   => tIntervslPQ(Lpp, Lqq.tail, acc ++ List((Lp.head, Lqq.head)))
        }
}
    val  lpp = lp.sortWith(_ < _) filter (e => e >= ti  && e<= lq.max)
    val  lqq = lq.sortWith(_ < _) filter (e => e >= ti  && e<= tf)
    tIntervslPQ(lpp, lqq)
}catch{case e: Exception => List.empty[(Long, Long)]}
} 
val mIntervalPQ = Memo.immutableHashMapMemo{intervalPQ _ tupled}
def intervalP(lp: List[Long],  ti: Long = 0, tf: Long = Long.MaxValue): List[(Long, Long)]={
    val lpp = lp.sortWith(_ < _) filter (e => e >= ti  && e<= tf)
    lpp zip lpp}
val mIntervalP  = Memo.immutableHashMapMemo{intervalP _ tupled}
def intervals[T](b: DataSet[(Long, T)], p: T => Boolean, q: T => Boolean = null, ti: Long = 0, tf: Long = Long.MaxValue)
=  q match{
        case null => mIntervalP(parse(b,p), ti, tf)
        case _    => mIntervalPQ(parse(b,p), parse(b,q), ti, tf)
    }
def bPQ[T](b: DataSet[(Long, T)], i: List[(Long, Long)])
= i.map(e => b.filter(x => (x._1 >= e._1) && (x._1 <= e._2)))
def Phi[T](b: DataSet[(Long, T)], ti: Long = 0, tf: Long = Long.MaxValue): Boolean = ???
def True[T](b: DataSet[(Long, T)], ti: Long = 0, tf: Long = Long.MaxValue): Boolean = true
def PhiAll[T](p: T => Boolean)(b: DataSet[(Long, T)], ti: Long, tf: Long): Boolean = {
        val bb = b.filter(e => (e._1 >= ti) && (e._1 <= tf))
        bb.map(e => p(e._2)).collect.forall(_ == true)
}
def PhiK[T](K: Long)(p: T => Boolean, q: T => Boolean = null)
    = {(b: DataSet[(Long, T)], ti: Long, tf: Long) 
    => intervals(b, p, q, ti, tf).map(e => (e._2 - e._1 >= K)).
    reduceOption(_ || _).getOrElse(false)}
def Neg[T](F: (DataSet[(Long, T)], Long, Long) => Boolean)
    ={(b: DataSet[(Long, T)], ti: Long, tf: Long) => !F(b, ti, tf)}
def Or[T](F1: (DataSet[(Long, T)], Long, Long) => Boolean, 
        F2: (DataSet[(Long, T)], Long, Long) => Boolean)
    = {(b: DataSet[(Long, T)], ti: Long, tf: Long) 
    => F1(b, ti, tf) || F2(b, ti, tf)}  
def U[T](p: T => Boolean, q: T => Boolean = null)
    (F1: (DataSet[(Long, T)], Long, Long) => Boolean,
    F2: (DataSet[(Long, T)], Long, Long) => Boolean)
    = {(b: DataSet[(Long, T)], ti: Long, tf: Long)
    => intervals(b, p, q, ti, tf).map(e => F1(b, ti, e._1) && F2(b, e._1, e._2)).
    reduceOption(_ || _).getOrElse(false)}                                 
def E[T](p: T => Boolean, q: T => Boolean = null)
    (F: (DataSet[(Long, T)], Long, Long) => Boolean)
    = {(b: DataSet[(Long, T)], ti: Long, tf: Long) 
    =>  val I = intervals(b, p, q, ti, tf); I.map(e => F(b, e._1, e._2)).
    reduceOption(_ || _).getOrElse(false)}
def E2[T](p: T => Boolean, q: T => Boolean = null)
        (F: (DataSet[(Long, T)], Long, Long) => Boolean)  = U(p, q)(True, F)
def A[T](p: T => Boolean, q: T => Boolean = {(e: T) => true})
        (F: (DataSet[(Long, T)], Long, Long) => Boolean)
    = {(b: DataSet[(Long, T)], ti: Long, tf: Long)
    =>  val I = intervals(b, p, q, ti, tf); I.map(e => F(b, e._1, e._2)).reduceOption(_ && _).getOrElse(false)}
def A2[T](p: T => Boolean, q: T => Boolean = null)
        (F: (DataSet[(Long, T)], Long, Long) => Boolean)  = Neg(E(p, q)(Neg(F)))

defined type T
defined function memoizeFnc
defined function apply
mApply: (DataSet[Tuple2[Long, Nothing]], Function1[Nothing, Boolean]) => DataSet[(Long, Boolean)] = scalaz.Memo$$$Lambda$112/0x0000000801377a40@7403ed42
defined function parse
defined function intervalPQ
mIntervalPQ: (List[Long], List[Long], Long, Long) => List[(Long, Long)] = scalaz.Memo$$$Lambda$112/0x0000000801377a40@4086e02
defined function intervalP
mIntervalP: (List[Long], Long, Long) => List[(Long, Long)] = scalaz.Memo$$$Lambda$112/0x0000000801377a40@5ba526d0
defined function intervals
defined function bPQ
defined function Phi
defined function True
defined function PhiAll
defined function PhiK
defined function Neg
defined function Or
defined function U
defined function E
defined function E2
defined function A
defined function A2

In [6]:
//type T = (Long, Int)
type TT = Int
val path = os.pwd / "csv"
val benv = ExecutionEnvironment.getExecutionEnvironment
val _10      = benv.readCsvFile[T](path+"/10.csv", fieldDelimiter = ";")

def Start: (TT => Boolean) = (e: TT) => {e == 1}
def Stop : (TT => Boolean) = (e: TT) => {e == 3}
def Cond : (TT => Boolean) = (e: TT) => {e == 2}

println(A(Start, Stop) (E(Cond) (True)) (_10, 0, Long.MaxValue))

true


defined type TT
path: os.package.pwd.ThisType = root/'Users/'bouao/'git/'ze2nb/'test/'csv
benv: ExecutionEnvironment = org.apache.flink.api.scala.ExecutionEnvironment@21facd61
_10: DataSet[(Long, Int)] = org.apache.flink.api.scala.DataSet@ff111f3
defined function Start
defined function Stop
defined function Cond

In [7]:
val _001in1M      = benv.readCsvFile[T](path+"/001in1M.csv", fieldDelimiter = ";")
println(A(Start, Stop) (A(Cond) (True)) (_001in1M, 0, Long.MaxValue))

true


_001in1M: DataSet[(Long, Int)] = org.apache.flink.api.scala.DataSet@57f98eb7

In [8]:
val _001in1M      = benv.readCsvFile[T](path+"/001in1M.csv", fieldDelimiter = ";")
println(A(Start, Stop) (PhiAll(Cond)) (_001in1M, 0, Long.MaxValue))

false


_001in1M: DataSet[(Long, Int)] = org.apache.flink.api.scala.DataSet@513d0baa

In [9]:
val _100in1M      = benv.readCsvFile[T](path+"/100in1M.csv", fieldDelimiter = ";")
println(A(Start, Stop) (PhiAll(Cond)) (_100in1M, 0, Long.MaxValue))

false


_100in1M: DataSet[(Long, Int)] = org.apache.flink.api.scala.DataSet@5b43e0b5

In [10]:
val _10      = benv.readCsvFile[T](path+"/10.csv", fieldDelimiter = ";")

_10: DataSet[(Long, Int)] = org.apache.flink.api.scala.DataSet@54a8f746

In [11]:
val f = (d : org.apache.flink.api.scala.DataSet[(Long, TT)]) => A(Start, Stop) (E(Cond) (True)) (d, 0, Long.MaxValue)

f: DataSet[(Long, TT)] => Boolean = <function1>

In [68]:
type TT = Int
val path = os.pwd / "csv"

val _10      = benv.readCsvFile[T](path+"/10.csv", fieldDelimiter = ";")
val _001in1M = benv.readCsvFile[T](path+"/001in1M.csv", fieldDelimiter = ";")
val _010in1M = benv.readCsvFile[T](path+"/010in1M.csv", fieldDelimiter = ";")
val _020in1M = benv.readCsvFile[T](path+"/020in1M.csv", fieldDelimiter = ";")
val _030in1M = benv.readCsvFile[T](path+"/030in1M.csv", fieldDelimiter = ";")
val _040in1M = benv.readCsvFile[T](path+"/040in1M.csv", fieldDelimiter = ";")
val _050in1M = benv.readCsvFile[T](path+"/050in1M.csv", fieldDelimiter = ";")
val _060in1M = benv.readCsvFile[T](path+"/060in1M.csv", fieldDelimiter = ";")
val _070in1M = benv.readCsvFile[T](path+"/070in1M.csv", fieldDelimiter = ";")
val _080in1M = benv.readCsvFile[T](path+"/080in1M.csv", fieldDelimiter = ";")
val _090in1M = benv.readCsvFile[T](path+"/090in1M.csv", fieldDelimiter = ";")
val _100in1M = benv.readCsvFile[T](path+"/100in1M.csv", fieldDelimiter = ";")


println("Formula performance 1: A(Start, Stop) (E(Cond) (True))")
println("Formula performance 2: A2(Start, Stop) (E2(Cond) (True))")
println("Both formulae check if eventually exists number 2 in any interval (1, 3)")
def Start: (TT => Boolean) = (e: TT) => {e == 1}
def Stop : (TT => Boolean) = (e: TT) => {e == 3}
def Cond : (TT => Boolean) = (e: TT) => {e == 2}
println("The stream example contains:")
println(_10.collect)
println("We will increase the number of this interval from 10 to 100 and mesure the elapsed time.")

val f = (d : org.apache.flink.api.scala.DataSet[(Long, TT)]) => A(Start, Stop) (E(Cond) (True)) (d, 0, Long.MaxValue)
val f2 = (d : org.apache.flink.api.scala.DataSet[(Long, TT)]) => A2(Start, Stop) (E2(Cond) (True)) (d, 0, Long.MaxValue)
val Time2Long = (e: Quantity[Double]) => e.value.toLong 

for(i <- 1 to 10){
     val list = List(_010in1M, _020in1M, _030in1M, _040in1M, _050in1M, _060in1M, _070in1M,
                    _080in1M, _090in1M, _100in1M).map(b => measure(f(b)))
     val list2 = List(_010in1M, _020in1M, _030in1M, _040in1M, _050in1M, _060in1M, _070in1M,
                    _080in1M, _090in1M, _100in1M).map(b => measure(f2(b)))                     
     println(list)
     println(list2)
     println((list.map(Time2Long(_)) zip list2.map(Time2Long(_))).map(e => e._2 - e._1).mkString("; "))
}


Formula performance 1: A(Start, Stop) (E(Cond) (True))
Formula performance 2: A2(Start, Stop) (E2(Cond) (True))
Both formulae check if eventually exists number 2 in any interval (1, 3)
The stream example contains:
Buffer((7,0), (8,0), (4,0), (5,2), (1,1), (2,0), (10,3), (6,0), (9,0), (3,0))
We will increase the number of this interval from 10 to 100 and mesure the elapsed time.
List(5881.2422 ms, 9839.1954 ms, 14255.8825 ms, 16507.5339 ms, 22480.811 ms, 81775.423 ms, 222913.4281 ms, 261493.8504 ms, 306852.022 ms, 365467.7957 ms)
List(44346.7055 ms, 82257.1553 ms, 119306.5816 ms, 142293.3314 ms, 195938.6269 ms, 240020.2545 ms, 283401.4091 ms, 101438.3251 ms, 294356.5598 ms, 131324.591 ms)
38465; 72418; 105051; 125786; 173458; 158245; 60488; -160055; -12496; -234143


: 

In [13]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import argonaut._
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
// init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import argonaut._

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
// init(offline=true)

// restrict the output height to avoid scrolling in output cells


In [14]:
import $ivy.`org.vegas-viz::vegas:0.3.11`
import vegas._
import vegas.data.External._

import $ivy.$                            

import vegas._

import vegas.data.External._

In [28]:
Vegas().
  withURL(Population).
  mark(Bar).
  addTransformCalculation("gender", """datum.sex == 2 ? "Female" : "Male"""").
  filter("datum.year == 2000").
  encodeColumn("age", Ord, scale=Scale(padding=4.0), axis=Axis(orient=Orient.Bottom, axisWidth=1.0, offset= -8.0)).
  encodeY("people", Quantitative, aggregate=AggOps.Sum, axis=Axis(title="population", grid=false)).
  encodeX("gender", Nominal, scale=Scale(bandSize = 6.0), hideAxis=true).
  encodeColor("gender", Nominal, scale=Scale(rangeNominals=List("#EA98D2", "#659CCA"))).
  configFacet(cell=CellConfig(strokeWidth = 0)).
  show

<iframe id="frame-vegas-aef4acec-b884-483d-ab46-77946f78e8bd" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-aef4acec-b884-483d-ab46-77946f78e8bd'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "column" : {
 "axis" : {
 "orient" : "bottom",
 "axisWidth" : 1.0,
 "offset" : -8.0
 },
 "scale" : {
 "padding" : 4.0
 },
 "field" : "age",
 "type" : "ordinal"
 },
 "x" : {
 "axis" : false,
 "scale" : {
 "bandSize" : 6.0
 },
 "field" : "gender",
 "type" : "nominal"
 },
 "y" : {
 "axis" : {
 "title" : "population",
 "grid" : false
 },
 "field" : "people",
 "type" : "quantitative",
 "aggregate" : "sum"
 },
 "color" : {
 "scale" : {
 "range" : [
 "#EA98D2",
 "#659CCA"
 ]
 },
 "field" : "gender",
 "type" : "nominal"
 }
 },
 "data" : {
 "url" : "https://vega.github.io/vega-editor/app/data/population.json"
 },
 "transform" : {
 "filter" : "datum.year == 2000",
 "calculate" : [
 {
 "field" : "gender",
 "expr" : "datum.sex == 2 ? \"Female\" : \"Male\""
 }
 ]
 },
 "config" : {
 "facet" : {
 "cell" : {
 "strokeWidth" : 0.0
 }
 }
 }
}
 }
 vg.embed("#vegas-aef4acec-b884-483d-ab46-77946f78e8bd", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

In [16]:
import $ivy.`org.vegas-viz::vegas-flink:0.3.11`
import vegas.flink.Flink._

import $ivy.$                                  

import vegas.flink.Flink._

In [67]:
Vegas().
    withData(Seq(
    Map("a" -> "A", "b" -> 28, "Type" -> "Recursive"), Map("a" -> "A2", "b" -> 55, "Type" -> "Direct"), 
    Map("a" -> "B", "b" -> 43, "Type" -> "Recursive"), Map("a" -> "B2", "b" -> 80, "Type" -> "Direct"),
    Map("a" -> "C", "b" -> 81, "Type" -> "Recursive"), Map("a" -> "C2", "b" -> 30, "Type" -> "Direct"),
    Map("a" -> "D", "b" -> 21, "Type" -> "Recursive"), Map("a" -> "D2", "b" -> 70, "Type" -> "Direct")
  )).
    mark(Bar).
    encodeColumn("a", Ordinal, scale=Scale(padding=4.0), axis=Axis(orient=Orient.Bottom, axisWidth=2.0, offset= 0.0)).
    encodeX("a", Ordinal, scale=Scale(bandSize = 3.0), hideAxis=true).
    encodeY("b", Quantitative, axis=Axis(title="Duration [ms]", grid=false)).
    encodeColor("Type", Nominal, scale=Scale(rangeNominals=List("#EA98D2", "#659CCA"))).
    configFacet(cell=CellConfig(strokeWidth = 0)).
    show

<iframe id="frame-vegas-35570f34-8785-45c2-99e6-eff50a682e23" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-35570f34-8785-45c2-99e6-eff50a682e23'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "column" : {
 "axis" : {
 "orient" : "bottom",
 "axisWidth" : 2.0,
 "offset" : 0.0
 },
 "scale" : {
 "padding" : 4.0
 },
 "field" : "a",
 "type" : "ordinal"
 },
 "x" : {
 "axis" : false,
 "scale" : {
 "bandSize" : 3.0
 },
 "field" : "a",
 "type" : "ordinal"
 },
 "y" : {
 "axis" : {
 "title" : "Duration [ms]",
 "grid" : false
 },
 "field" : "b",
 "type" : "quantitative"
 },
 "color" : {
 "scale" : {
 "range" : [
 "#EA98D2",
 "#659CCA"
 ]
 },
 "field" : "Type",
 "type" : "nominal"
 }
 },
 "data" : {
 "values" : [
 {
 "a" : "A",
 "b" : 28,
 "Type" : "Recursive"
 },
 {
 "a" : "A2",
 "b" : 55,
 "Type" : "Direct"
 },
 {
 "a" : "B",
 "b" : 43,
 "Type" : "Recursive"
 },
 {
 "a" : "B2",
 "b" : 80,
 "Type" : "Direct"
 },
 {
 "a" : "C",
 "b" : 81,
 "Type" : "Recursive"
 },
 {
 "a" : "C2",
 "b" : 30,
 "Type" : "Direct"
 },
 {
 "a" : "D",
 "b" : 21,
 "Type" : "Recursive"
 },
 {
 "a" : "D2",
 "b" : 70,
 "Type" : "Direct"
 }
 ]
 },
 "config" : {
 "facet" : {
 "cell" : {
 "strokeWidth" : 0.0
 }
 }
 }
}
 }
 vg.embed("#vegas-35570f34-8785-45c2-99e6-eff50a682e23", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">